# Menganalisis risiko gagal bayar peminjam

Tugasmu adalah menyiapkan laporan untuk divisi kredit suatu bank. Kamu akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman. Pihak bank sudah memiliki beberapa data mengenai kelayakan kredit nasabah.

Laporanmu akan dipertimbangkan pada saat membuat **penilaian kredit** untuk calon nasabah. **Penilaian kredit** digunakan untuk mengevaluasi kemampuan calon peminjam untuk melunasi pinjaman mereka.


# Konten

- [Pendahuluan](#intro)
- [Buka file data dan baca informasi umumnya]()
- [Soal 1. Eksplorasi data]()
- [Transformasi Data]()
    - [3.1 Memperbaiki nilai yang hilang di total_income]()
    - [3.2  Memperbaiki nilai di days_employed]()
- [4  Pengkategorian data]()
- [5  Memeriksa hipotesis]()
    - [Kesimpulan]()

# Pendahuluan

Sebuah bank sudah memberikan pinjaman kepada nasabah ke berbagai kalangan dan dengan berbagai tujuan. Pihak bank ingin melakukan analisa apa penyebab nasabah gagal melakukan pembayaran, yang mana hasil analisa ini akan digunakan sebagai penilaian kredit untuk calon nasabah. 

Pihak bank menyediakan data nasabah yang melakukan peminjaman. Dan kita perlu melakukan analisa terlebih dahulu terhadap data yang disediakan tersebut sebelum diproses dan diambil kesimpulan. Analisa ini dilakukan untuk memastikan bahwa data yang diambil dan diproses merupakan data yang layak untuk dijadikan pengambilan keputusan. Setelah data diproses, perlu ada pembuktian terhadap hipotesis yang diberikan.

Kesimpulan akhir dari hasil analisa akan menjadi salah satu komponen penilaian yang dapat dilakukan oleh bank untuk melihat potensi nasabah, serta persetujuan untuk memberikan peminjaman ke nasabah. 

## Buka file data dan baca informasi umumnya.

In [1]:
import pandas as pd

In [2]:
credit_scoring = pd.read_csv('datasets/credit_scoring_eng.csv')

## Soal 1. Eksplorasi data

**Deskripsi data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - status perkawinan
- `family_status_id` - pengidentifikasi untuk status perkawinan nasabah
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah pernah melakukan gagal bayar pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman



In [3]:
credit_scoring.count()

children            21525
days_employed       19351
dob_years           21525
education           21525
education_id        21525
family_status       21525
family_status_id    21525
gender              21525
income_type         21525
debt                21525
total_income        19351
purpose             21525
dtype: int64

In [4]:
credit_scoring.shape

(21525, 12)

In [5]:
credit_scoring.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Dari 10 data teratas, ada 2 hal yang perlu diperhatikan, yaitu:

Pertama, terdapat keanehan pada kolom days_employed, dimana value berisi dibawah 0. Sedangkan pengalaman kerja seharusnya minimal adalah 0. 

Kedua, Kolom education juga ada inkonsistensi data. Meskipun education_id memiliki value yang konsisten dengan education, namun penulisan di kolom education perlu diperbaiki dan diseragamkan.

In [6]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Total terdapat 21525 row, dan dari info, ada 2 kolom yang jumlah nya tidak mencapai 21525 yaitu days_employed dan total_income. Hal ini menunjukan bahwa nilai yang hilang hanya terdapat pada 2 kolom tersebut.

In [7]:
credit_scoring_na = credit_scoring.loc[credit_scoring['days_employed'].isna()]
credit_scoring_na.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


Tampaknya, nilai yang hilang pada days_employed juga berkaitan dengan nilai pada total_income yang juga hilang. Selain dengan kolom total_income, tidak terlihat pola atau keterkaitan nilai yang hilang pada kolom days_employed dengan kolom lain.

In [8]:
credit_scoring_na[credit_scoring_na['total_income'].isna()].shape


(2174, 12)

**Kesimpulan sementara**

Total jumlah baris adalah 21525. Jumlah baris yang hilang adalah 2174. Dan bila diperhatikan setiap row dengan nilai yang hilang pada kolom days_employed juga berlaku nilai yang hilang pada kolom total_income . Bila dibandingkan dengan hasil sebelumnya, maka total baris yang memiliki nilai yang lengkap adalah 19351. Artinya jumlah baris yang difilter sesuai dengan jumlah nilai yang hilang. 

Persentase nilai yang hilang sebesar 10%. Angka ini terlihat sangat besar dan bisa saja mempengaruhi hasil dari penelitian.

Berikutnya, akan dilakukan pengecekan terhadap beberapa data kategorikal yang bisa saja memiliki sebuah pola pada nilai yang hilang

In [9]:
credit_scoring[(credit_scoring['days_employed'].isna() == False)]['education_id'].unique()


array([0, 1, 2, 3, 4], dtype=int64)

In [10]:
credit_scoring_na['education_id'].value_counts()


education_id
1    1540
0     544
2      69
3      21
Name: count, dtype: int64

**Kemungkinan penyebab hilangnya nilai dalam data**

Nilai yang hilang mungkin saja disebabkan tingkat pendidikan dari nasabah. Namun, dilihat dari distribusinya, nilai yang hilang hampir terjadi di semua tingkat pendidikan. 

In [11]:
credit_scoring['education_id'].value_counts()


education_id
1    15233
0     5260
2      744
3      282
4        6
Name: count, dtype: int64

**Kesimpulan sementara**

Hasil distribusi di seluruh dataset menampilkan tingkat pendidikan nasabah sebagian besar berada pada id 0,1,2, dan 3, yang mana kesemua id tersebut juga memiliki data yang hilang. Sehingga belum bisa dipastikan bahwa penyebab nilai yang hilang dikarenakan tingkat pendidikan tertentu pada nasabah.

In [12]:
credit_scoring_na['family_status_id'].value_counts()

family_status_id
0    1237
1     442
4     288
3     112
2      95
Name: count, dtype: int64

**Kesimpulan sementara**

Jika diperhatikan melalui family_status_id, nilai yang hilang juga berlaku di semua status perkawinan nasabah.

In [13]:
credit_scoring_na['income_type'].value_counts()

income_type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: count, dtype: int64

**Kesimpulan**

Dari identifikasi yang sudah dilakukan, tidak ditemukan adanya pola tertentu yang menyebabkan nilai hilang. Identifikasi terakhir melalui jenis pendapatan nasabah juga tidak ditemukan adanya anomali, karena nilai yang hilang berlaku untuk jenis income yang seharusnya sudah bekerja.

Karena data yang hilang adalah data kuantitatif, maka data yang diisi adalah rata-rata atau median berdasarkan tingkat pendidikan dan status perkawinan nasabah.

## Transformasi data


In [14]:
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [15]:
def replace_wrong_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        credit_scoring['education'] = credit_scoring['education'].replace(wrong_value, correct_value)
        
replace_wrong_values(["bachelor's degree", "BACHELOR'S DEGREE"], "Bachelor's Degree")
replace_wrong_values(["secondary education", "SECONDARY EDUCATION"], "Secondary Education")
replace_wrong_values(["some college", "SOME COLLEGE"], "Some College")
replace_wrong_values(["primary education", "PRIMARY EDUCATION"], "Primary Education"),
replace_wrong_values(["GRADUATE DEGREE", "graduate degree"], "Graduate Degree")

In [16]:
credit_scoring['education'].unique()


array(["Bachelor's Degree", 'Secondary Education', 'Some College',
       'Primary Education', 'Graduate Degree'], dtype=object)

In [17]:
credit_scoring['children'].value_counts()

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64

Ada 2 hal yang perlu diperbaiki pada kolom children. Pertama, jelas data dengan value -1. Sangat tidak mungkin nasabah memiliki anak dibawah 0. Untuk data dengan value -1 di kolom children ini bisa diubah dengan 2 pilihan, dengan median atau diubah ke 0. Namun, lebih masuk akal bila value -1 ini diubah menjadi 0, karena bisa saja nasabah memang belum memiliki anak, namun terdapat kesalahan pada sistem. 
Masalah kedua adalah data dengan value 20. Ini bisa saja disebabkan kesalahan penulisan yang dilakukan oleh nasabah. Dalam hal ini, value akan diisi menggunakan nilai median. Karena terdapat jarak yang signifikan antara jumlah nasabah yang tidak memiliki anak dengan yang memiliki anak. 

In [18]:
children_median = credit_scoring['children'].median()

credit_scoring.loc[credit_scoring['children'] < 0, 'children'] = 1
credit_scoring.loc[credit_scoring['children'] == 20, 'children'] = 2

In [19]:
credit_scoring['children'].value_counts()


children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: count, dtype: int64

[Periksa data dalam kolom `days_employed`. Pertama-tama, pikirkan tentang masalah apa yang mungkin ada pada kolom tersebut, serta pikirkan juga apa yang mungkin ingin kamu periksa dan bagaimana kamu akan melakukannya.]

In [20]:
credit_scoring_below_zero = credit_scoring[(credit_scoring['days_employed'] < 0) & (credit_scoring['total_income'] > 0)]
credit_scoring_below_zero_percentage = credit_scoring_below_zero.shape[0]/credit_scoring.shape[0]
credit_scoring_below_zero_percentage

0.7389547038327526

Jumlah data yang bermasalah mencapai 15906 baris, yang berarti terdapat masalah pada 73% baris dari total data normal. Jika dilihat, ini bisa saja terdapat kesalahan pada perhitungan dimana nilai seharusnya positif, bukan negatif. Sehingga kita perlu mengubah nilai ini menjadi positif.

In [21]:
for index in range(len(credit_scoring)):
    if (credit_scoring['days_employed'][index] < 0) & (credit_scoring['total_income'][index] > 0):
        credit_scoring.loc[index, 'days_employed'] = credit_scoring['days_employed'][index] * -1

In [22]:
credit_scoring[(credit_scoring['days_employed'] < 0) & (credit_scoring['total_income'] > 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Umumnya usia seseorang pasti diatas 0, atau bukan minus. Sehingga proses filternya mencari usia yang dibawah atau sama dengan 0.

In [23]:
credit_scoring[credit_scoring['dob_years'] <= 0]['dob_years'].value_counts()

dob_years
0    101
Name: count, dtype: int64

Untuk nilai 0 pada kolom dob_years, maka nilai ini perlu diubah ke nilai rata-rata pada kolom dob_years. Karena tidak ada outlier signifikan pada umur. 

In [24]:
credit_scoring.loc[credit_scoring['dob_years'] <= 0, 'dob_years'] = int(credit_scoring['dob_years'].median()) #.round().astype('int')

In [25]:
credit_scoring[credit_scoring['dob_years'] <= 0]['dob_years'].value_counts()

Series([], Name: count, dtype: int64)

In [26]:
credit_scoring['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [27]:
credit_scoring['gender'].value_counts()

gender
F      14236
M       7288
XNA        1
Name: count, dtype: int64

In [28]:
credit_scoring = credit_scoring[credit_scoring['gender'] != 'XNA']

In [29]:
credit_scoring['gender'].unique()

array(['F', 'M'], dtype=object)

In [30]:
credit_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Terdapat 71 data duplikat pada dataframe. Dan 71 data ini perlu dihapus.

In [31]:
credit_scoring[credit_scoring.duplicated()].count()

children            72
days_employed        0
dob_years           72
education           72
education_id        72
family_status       72
family_status_id    72
gender              72
income_type         72
debt                72
total_income         0
purpose             72
dtype: int64

In [32]:
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)

In [33]:
credit_scoring[credit_scoring.duplicated()].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [34]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      19350 non-null  float64
 11  purpose           21452 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Data duplikat yang dihapus berjumlah 71, dan hanya sebesar 0,3% dari keseluruhan dataset.

# Bekerja dengan nilai yang hilang

Untuk mengurangi duplikasi, inkonsistensi dan size data, maka pada kolom education, education id dan famlily_status, family_status_id perlu dibuat dictionary pada 2 kolom tersebut. Dan mungkin bisa diterapkan juga pada kolom income_type.

In [35]:
education = credit_scoring[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
family_status = credit_scoring[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
income_type = credit_scoring[['income_type']].drop_duplicates().reset_index(drop=True)                             
income_type.insert(0, 'income_type_id', range(0,0+len(income_type)))

### Memperbaiki nilai yang hilang di `total_income`

Ada 2 kolom dengan nilai yang hilang, yaitu days_employed dan total_income. Nilai yang hilang pada dedua kolom ini perlu diperbaiki. Untuk memperbaiki ini perlu membagi pendapatan ke beberapa kategori usia, dan menghitung rata-rata pada setiap kategori usia. 


In [36]:
def age_group(row):
    age = int(row['dob_years'])
    
    if age < 18:
        return 'children'
    if age < 45:
        return 'adult'
    if age < 64:
        return 'middle age'
    return 'retired'

    

In [37]:
row_values = [20]
row_columns = ['dob_years']
row = pd.Series(data=row_values, index=row_columns)
age_group(row)

'adult'

In [38]:
credit_scoring['age_group'] = credit_scoring.apply(age_group, axis=1)


In [39]:
credit_scoring.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,Bachelor's Degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,Secondary Education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,Secondary Education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,Secondary Education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle age
5,0,926.185831,27,Bachelor's Degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult
6,0,2879.202052,43,Bachelor's Degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50,Secondary Education,1,married,0,M,employee,0,21731.829,education,middle age
8,2,6929.865299,35,Bachelor's Degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,Secondary Education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [40]:
credit_scoring_non_missing = credit_scoring[(credit_scoring['days_employed'].isna() == False) & (credit_scoring['total_income'].isna() == False)]

In [41]:
credit_scoring_non_missing.groupby('age_group')['total_income'].mean()

age_group
adult         27657.581380
middle age    26159.377907
retired       22489.034154
Name: total_income, dtype: float64

In [42]:
credit_scoring_non_missing.groupby('age_group')['total_income'].median()

age_group
adult         24046.1455
middle age    22737.8935
retired       18930.2065
Name: total_income, dtype: float64

Karakteristik yang paling menentukan pendapatan adalah kelompok umur. Karena pendapatan setiap nasabah berbeda-beda, dan ada potensi terdapat outlier yang signifikan, maka untuk mengisi nilai yang hilang lebih baik menggunakan nilai median.


In [43]:
def replace_missing_value_total_income(column, replaced_by):    
    credit_scoring[column] = credit_scoring[column].fillna(credit_scoring_non_missing.groupby('age_group').get_group('adult')['total_income'].median())          

In [44]:
credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,Bachelor's Degree,0,married,0,F,employee,0,40620.102,purchase of the house,adult
1,1,4024.803754,36,Secondary Education,1,married,0,F,employee,0,17932.802,car purchase,adult
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,adult
3,3,4124.747207,32,Secondary Education,1,married,0,M,employee,0,42820.568,supplementary education,adult
4,0,340266.072047,53,Secondary Education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,middle age
5,0,926.185831,27,Bachelor's Degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,adult
6,0,2879.202052,43,Bachelor's Degree,0,married,0,F,business,0,38484.156,housing transactions,adult
7,0,152.779569,50,Secondary Education,1,married,0,M,employee,0,21731.829,education,middle age
8,2,6929.865299,35,Bachelor's Degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,adult
9,0,2188.756445,41,Secondary Education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,adult


In [45]:
replace_missing_value_total_income('total_income', credit_scoring_non_missing['total_income'].median())

In [46]:
credit_scoring[credit_scoring['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


In [47]:
credit_scoring.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
 12  age_group         21452 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Memperbaiki nilai di `days_employed`

In [48]:
credit_scoring_non_missing.groupby('age_group')['days_employed'].median()

age_group
adult           1438.446525
middle age      4140.545518
retired       359057.281090
Name: days_employed, dtype: float64

In [49]:
credit_scoring_non_missing.groupby('age_group')['days_employed'].mean()

age_group
adult           5153.814550
middle age    120198.678019
retired       308323.844132
Name: days_employed, dtype: float64

Nilai yang hilang pada days_employed akan digantikan dengan nilai median di masing-masing kategori. Karena terdapat perbedaan signifikan antara nilai terendah dan nilai tertinggi.

In [50]:
def days_employed_median(age_group):
    return credit_scoring_non_missing.groupby('age_group').get_group(age_group)['days_employed'].median()

        

In [51]:
days_employed_median('adult')


1438.4465246413963

In [52]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(days_employed_median('adult'))

In [53]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  float64
 11  purpose           21452 non-null  object 
 12  age_group         21452 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


## Pengkategorian data



In [54]:
credit_scoring['purpose']

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21447       housing transactions
21448          purchase of a car
21449                   property
21450          buying my own car
21451               to buy a car
Name: purpose, Length: 21452, dtype: object

In [55]:
credit_scoring['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Berdasarkan tujuan dari peminjaman, sebenarnya dapat dikelompokan menjadi beberapa kategori. Disini terlihat banyak sekali tujuan karena memang penulisannya yang berbeda, tapi tujuannya sama.

In [56]:
def cat_purpose(row):
    purpose = row['purpose']
    if 'house' in purpose or 'estate' in purpose or 'home' in purpose:
            return 'real_estate'
    elif 'car' in purpose:
            return 'car'
    elif 'wedding' in purpose:
            return 'wedding'
    else:
            return 'university'
credit_scoring['cat_purpose'] = credit_scoring.apply(cat_purpose, axis=1)

In [57]:
credit_scoring['cat_purpose'].value_counts()

cat_purpose
university     8456
real_estate    6367
car            4306
wedding        2323
Name: count, dtype: int64

In [58]:
credit_scoring['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21447    35966.698
21448    24959.969
21449    14347.610
21450    39054.888
21451    13127.587
Name: total_income, Length: 21452, dtype: float64

In [59]:
credit_scoring['total_income'].describe()

count     21452.000000
mean      26518.674634
std       15668.982440
min        3306.762000
25%       17217.441750
50%       24046.145500
75%       31328.693750
max      362496.645000
Name: total_income, dtype: float64

Total income perlu dikelompokan karena kemungkinan nasabah gagal bayar bisa disebabkan oleh pendapatan nasabah.

In [60]:
def cat_total_income(row):
    total_income = row['total_income']
    if total_income < 15000:
        return 'low'
    if total_income < 30000:
        return 'medium'
    if total_income < 45000:
        return 'high'
    return 'very high'


In [61]:
credit_scoring['cat_total_income'] = credit_scoring.apply(cat_total_income, axis=1)

In [62]:
credit_scoring['cat_total_income'].value_counts()

cat_total_income
medium       11791
high          4059
low           3743
very high     1859
Name: count, dtype: int64

## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [63]:
credit_scoring[credit_scoring['debt'] == 1]['children'].unique()

credit_scoring.pivot_table(index='children', values='debt', aggfunc='mean')

,debt
children,
0,0.075449
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


**Kesimpulan**

Dari persentase, semakin sedikit jumlah anak, semakin banyak nasabah yang gagal bayar pinjaman. Sebaliknya, semakin banyak anak semakin sedikit jga nasabah yang gagal bayar pinjaman. Namun, hal ini mungkin saja disebabkan sangat banyaknya nasabah yang belum memiliki anak ketimbang yang sudah mempunyai anak.


**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [64]:
credit_scoring[credit_scoring['debt'] == 1]['family_status'].unique()

credit_scoring.pivot_table(index='family_status', values='debt', aggfunc='mean')

,debt
family_status,
civil partnership,0.093517
divorced,0.071130
married,0.075452
unmarried,0.097509
widow / widower,0.065693


**Kesimpulan**

Terlihat dari nasabah yang belum menikah justru memiliki persentase gagal bayar lebih tinggi ketimbang yang sudah menikah. Tetapi sebaliknya, nasabah yang berstatus cerai dimana statusnya sendiri, lebih sedikit persentase dalam gagal bayar. 


**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [65]:
credit_scoring.groupby('cat_total_income')['debt'].value_counts()

credit_scoring.pivot_table(index='cat_total_income', values='debt', aggfunc='mean')


,debt
cat_total_income,
high,0.074895
low,0.079615
medium,0.085404
very high,0.071006


**Kesimpulan**

Total pendapatan tidak menjamin nasabah mengalami gagal bayar. Terlihat dari persentase yang tidak jauh berbeda antara nasabah yang memiliki pendapatan rendah dengan pendapatan paling tinggi sekalipun. 

**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [66]:
credit_scoring.pivot_table(index='cat_purpose', values='debt', aggfunc='mean')

,debt
cat_purpose,
car,0.093590
real_estate,0.072719
university,0.081481
wedding,0.080069


**Kesimpulan**

Nasabah yang melakukan peminjaman dengan tujuan untuk melakukan pembelian barang tersier (mobil) lebih berpotensi gagal bayar.


# Kesimpulan umum 

Saat proses analisis data, ada beberapa data yang memang memiliki value yang tidak masuk akal. Seperti jumlah anak dibawah 0, dan bahkan mencapai 20. Untuk data ini perlu dipelajari lebih lanjut apakah karena kesalahan penulisan, kesalahan program atau kesalahan lainnya. Dalam analisis kali ini, data anak dibawah 0 diubah menjadi positif, contoh -1 menjadi 1, dan 20 menjadi 2. 

Selain jumlah anak, juga terdapat masalah pada jumlah hari kerja nasabah. Data ini tidak terlalu berkaitan dengan tujuan nasabah melakukan peminjaman ataupun penyebab gagal bayar. Masalah lainnya adalah kesalahan penulisan pada education, family status, yang tidak konsisten, sehingga sulit untuk mengkategorikan apakah masalah gagal bayar disebabkan oleh education atau family status.

Masalah berikutnya, mengenai nilai yang hilang pada kolom jumlah hari kerja dan pendapatan nasabah. Dimana jumlah nilai yang hilang seragam di kedua kolom ini. Jika diperhatikan, pendapatan nasabah bisa menjadi salah 1 alasan penyebab gagal nya nasabah dalam membayar pinjaman. Sehingga data yang hilang di kolom pendapatan perlu diisi sesuai dengan rata-rata pendapatan seluruh nasabah.

Terakhir mengenai 71 data yang terduplikasi. Untuk data duplikat, dapat dihapus karena tidak adanya perbedaan dengan data yang sudah ada. 

Dari hasil analisis, semakin sedikit anak, peluang nasabah gagal bayar juga semakin kecil. Sehingga terdapat kemungkinan nasabah dengan lebih banyak anak mengalami gagal bayar. Selain itu, faktor tujuan peminjaman untuk kebutuhan tersier juga memiliki potensi nasabah gagal bayar. Seperti nasabah yang melakukan peminjaman untuk tujuan pembelian mobil, memiliki persentase gagal bayar paling tinggi.  Sedangkan dari faktor lain seperti perkawinan, tingkat pendapatan, tidak terdapat jawaban pasti bahwa faktor tersebut dapat mempengaruhi nasabah dalam gagal melakukan pembayaran.
